## Predicting the stock market

Let's first import the necessary libraries!

In [1]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

C:\Users\SPN\anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


## Getting The Data

Having imported the appropriate tools, we will get market data from Yahoo Finance, a free online source, using pandas.
We'll be using this dataset to develop a predictive model. We'll train the model with data from 1971-2015, and try to make predictions from 2016-2019.

The columns of the dataset are:

    Date -- The date of the record.
    Open -- The opening price of the day (when trading starts).
    High -- The highest trade price during the day.
    Low -- The lowest trade price during the day.
    Close -- The closing price for the day (when trading is finished).
    Volume -- The number of shares traded.
    Adj Close -- The daily closing price, adjusted retroactively to include any corporate actions.


For more information about the columns head over to [this site](https://www.investopedia.com/terms/a/adjusted_closing_price.asp).

In [2]:
# Define the instruments to download. We would like to see the S&P500 index.
ticker = '^GSPC'

# We would like all available data from 01/01/1971 until 12/31/2019.
start_date = '1971-01-01'
end_date = '2019-12-31'

# Use pandas_reader.data.DataReader to load the desired data. 
df = data.DataReader('^GSPC', 'yahoo', start_date, end_date)

df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
1971-01-04,92.190002,90.639999,92.150002,91.150002,10010000,91.150002
1971-01-05,92.279999,90.690002,91.150002,91.800003,12600000,91.800003
1971-01-06,93.000000,91.500000,91.800003,92.349998,16960000,92.349998
1971-01-07,93.260002,91.750000,92.349998,92.379997,16460000,92.379997
1971-01-08,93.019997,91.599998,92.379997,92.190002,14100000,92.190002


In [3]:
#check for null values
df.isnull().sum()

High         0
Low          0
Open         0
Close        0
Volume       0
Adj Close    0
dtype: int64

In [4]:
#have a look at the index column- especially the data type
df.index

DatetimeIndex(['1971-01-04', '1971-01-05', '1971-01-06', '1971-01-07',
               '1971-01-08', '1971-01-11', '1971-01-12', '1971-01-13',
               '1971-01-14', '1971-01-15',
               ...
               '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20',
               '2019-12-23', '2019-12-24', '2019-12-26', '2019-12-27',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', name='Date', length=12359, freq=None)

In [5]:
#check the data types of all the other columns
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12359 entries, 1971-01-04 to 2019-12-31
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       12359 non-null  float64
 1   Low        12359 non-null  float64
 2   Open       12359 non-null  float64
 3   Close      12359 non-null  float64
 4   Volume     12359 non-null  int64  
 5   Adj Close  12359 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 675.9 KB


## Generating Indicators

Datasets taken from the stock market need to be handled differently than datasets from other sectors when it comes time to make predictions. In a normal machine learning exercise, we treat each row as independent. Stock market data is sequential, and each observation comes a day after the previous observation. Thus, the observations are not all independent, and you can't treat them as such.

This means you have to be extra careful to not inject "future" knowledge into past rows when you do training and prediction. Injecting future knowledge will make our model look good when you're training and testing it, but will make it fail in the real world. This is how many algorithmic traders lose money.

The time series nature of the data means that can generate indicators to make our model more accurate. For instance, you can create a new column that contains the average price of the last 10 trades for each row. This will incorporate information from multiple prior rows into one, and will make predictions much more accurate.

When you do this, you have to be careful not to use the current row in the values you average. You want to teach the model how to predict the current price from historical prices. If you include the current price in the prices you average, it will be equivalent to handing the answers to the model upfront, and will make it impossible to use in the "real world", where you don't know the price upfront.

Here are some indicators that are interesting to generate for each row:
* The average price from the past 5 days.
* The average price for the past 30 days.
* The average price for the past 365 days.
* The ratio between the average price for the past 5 days, and the average price for the past 365 days.
* The standard deviation of the price over the past 5 days.
* The standard deviation of the price over the past 365 days.
* The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.

For this analysis we are going to pick only 3 indicators:
* The average price from the past 5 days.
* The average price for the past 365 days.
* The ratio between the average price for the past 5 days, and the average price for the past 365 days.
* The standard deviation of the price over the past 5 days.
* The standard deviation of the price over the past 365 days.
* The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.

In [6]:
#compute the average price from the past 5 days for each row
data_mean_5day = df.Close.rolling(5).mean().shift(1)
# Compute the average price for the past 365 days.
data_mean_365day = df.Close.rolling(365).mean().shift(1)
# Compute the ratio between the average price for the past 5 days, and the average price for the past 365 days.
data_mean_ratio = data_mean_5day/data_mean_365day

# Compute the standard deviation of the price over the past 5 days.
data_std_5day = df.Close.rolling(5).std().shift(1)
# Compute the standard deviation of the price over the past 365 days.
data_std_365day = df.Close.rolling(365).std().shift(1)
# Compute the ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.
data_std_ratio = data_std_5day/data_std_365day

df['data_mean_5day'] = data_mean_5day
df['data_mean_365day'] = data_mean_365day
df['data_mean_ratio'] = data_mean_ratio
df['data_std_5day'] = data_std_5day
df['data_std_365day'] = data_std_365day
df['data_std_ratio'] = data_std_ratio

df.head(10)

,High,Low,Open,Close,Volume,Adj Close,data_mean_5day,data_mean_365day,data_mean_ratio,data_std_5day,data_std_365day,data_std_ratio
Date,,,,,,,,,,,,
1971-01-04,92.190002,90.639999,92.150002,91.150002,10010000,91.150002,NaN,NaN,NaN,NaN,NaN,NaN
1971-01-05,92.279999,90.690002,91.150002,91.800003,12600000,91.800003,NaN,NaN,NaN,NaN,NaN,NaN
1971-01-06,93.000000,91.500000,91.800003,92.349998,16960000,92.349998,NaN,NaN,NaN,NaN,NaN,NaN
1971-01-07,93.260002,91.750000,92.349998,92.379997,16460000,92.379997,NaN,NaN,NaN,NaN,NaN,NaN
1971-01-08,93.019997,91.599998,92.379997,92.190002,14100000,92.190002,NaN,NaN,NaN,NaN,NaN,NaN
1971-01-11,92.669998,90.989998,92.190002,91.980003,14720000,91.980003,91.974001,NaN,NaN,0.515295,NaN,NaN
1971-01-12,93.279999,91.629997,91.980003,92.720001,17820000,92.720001,92.140001,NaN,NaN,0.247687,NaN,NaN
1971-01-13,93.660004,91.879997,92.720001,92.559998,19070000,92.559998,92.324001,NaN,NaN,0.272451,NaN,NaN
1971-01-14,93.360001,91.669998,92.559998,92.800003,17600000,92.800003,92.366000,NaN,NaN,0.292881,NaN,NaN


In [7]:
# Use the dropna method to remove any rows with NaN values
df = df.dropna()
df.head()

,High,Low,Open,Close,Volume,Adj Close,data_mean_5day,data_mean_365day,data_mean_ratio,data_std_5day,data_std_365day,data_std_ratio
Date,,,,,,,,,,,,
1972-06-12,107.919998,106.290001,106.860001,107.010002,13390000,107.010002,107.764000,100.875836,1.068284,0.771122,4.869836,0.158347
1972-06-13,108.029999,106.379997,107.010002,107.550003,15710000,107.550003,107.402000,100.919288,1.064237,0.542374,4.853546,0.111748
1972-06-14,109.150002,107.379997,107.550003,108.389999,18320000,108.389999,107.270001,100.962438,1.062474,0.338601,4.842248,0.069926
1972-06-15,109.519997,107.779999,108.389999,108.440002,16940000,108.440002,107.418001,101.006383,1.063477,0.603961,4.836653,0.124872
1972-06-16,108.940002,107.540001,108.440002,108.360001,13010000,108.360001,107.650002,101.050383,1.065310,0.744211,4.831009,0.154049


## Splitting Up The Data

In [8]:
# Generate two new dataframes to use in making our algorithm. 
# train should contain any rows in the data with a date less than 2016-01-01. 
# test should contain any rows with a date greater than or equal to 2016-01-01.
df_train = df[df.index < datetime(year=2016, month=1, day=1)]
df_test = df[df.index >= datetime(year=2016, month=1, day=1)]

## Making Predictions: Training and Testing

Now, we will define an error metric, train a model using the train data, and make predictions on the test data.

It's recommended to use Mean Absolute Error, also called MAE, as an error metric, because it will show you how "close" you were to the price in intuitive terms. Mean Squared Error, or MSE, is an alternative that is more commonly used, but makes it harder to intuitively tell how far off you are from the true price because it squares the error.

In [9]:
from sklearn.linear_model import LinearRegression
#Initialize an instance of the LinearRegression class.
model = LinearRegression()
# Leave out all of the original columns (Close, High, Low, Open, Volume, Adj Close, Date) when training the model.
# These all contain knowledge of the future that we don't want to feed the model
features = ['data_mean_5day', 'data_mean_365day', 'data_mean_ratio', 'data_std_5day', 'data_std_365day', 'data_std_ratio']
X = df_train[features]
X_test = df_test[features]
y = df_train.Close
y_test = df_test.Close

#Train a linear regression model, using the train DataFrame
# Use the Close column as the target: 
model.fit(X, y)
#Make predictions for the Close column of the test data, using the same columns for training as you did with train.
pred = model.predict(X_test)
#Pick an error metric, MAE, and compute it.
MAE = sum(abs(pred - y_test))/len(pred)
print(MAE)
print(model.score(X, y))


20.892707743153323
0.9994306654445108


## Recommendations

For future work we will try to improve the error of this model (significantly). 
 
Here are some indicators that might be helpful to compute:
- The average volume over the past five days.
- The average volume over the past year.
- The ratio between the average volume for the past five days, and the average volume for the past year.
- The standard deviation of the average volume over the past five days.
- The standard deviation of the average volume over the past year.
- The ratio between the standard deviation of the average volume for the past five days, and the standard deviation of the average volume for the past year.
- The year component of the date.
- The ratio between the lowest price in the past year and the current price.
- The ratio between the highest price in the past year and the current price.
- The month component of the date.
- The day of week.
- The day component of the date.
- The number of holidays in the prior month.


There's a lot of improvement still to be made on the indicator side, and we urge you to think of better indicators that you could use for prediction. We can also make significant structural improvements to the algorithm, and pull in data from other sources.

* Accuracy would improve greatly by making predictions only one day ahead. For example, train a model using data from 1951-01-03 to 2013-01-02, make predictions for 2013-01-03, and then train another model using data from 1951-01-03 to 2013-01-03, make predictions for 2013-01-04, and so on. This more closely simulates what you'd do if you were trading using the algorithm.

   We can also improve the algorithm used significantly. Try other techniques, like a random forest, and see if they perform better.

  We can also incorporate outside data, such as the weather in New York City (where most trading happens) the day before, and the amount of Twitter activity around certain stocks.

  We can also make the system real-time by writing an automated script to download the latest data when the market closes, and make predictions for the next day.

    Finally, we can make the system "higher-resolution". We're currently making daily predictions, but we could make hourly, minute-by-minute, or second by second predictions. This will require obtaining more data, though. We could also make predictions for individual stocks instead of the S&P500.


#### And that's it for this project.